# 0.2.0 Simple topic identification

In [105]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:
import pandas as pd
from matplotlib import pyplot as plt
import re

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from gensim.corpora.dictionary import Dictionary

from gensim.models.tfidfmodel import TfidfModel

import itertools
from collections import defaultdict

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [107]:
import sys
sys.path.append("../") 

from utils.info import article, articles
import utils.paths as path
from utils.paths2 import direcciones, direcciones_cursos

## 0.2.2 Bag-of-words picker

In [108]:
Counter(word_tokenize("""The cat is in the box. The cat box."""))

Counter({'The': 2, 'cat': 2, 'is': 1, 'in': 1, 'the': 1, 'box': 2, '.': 2})

## 0.2.3 Building a Counter with bag-of-words

In [109]:
# Tokenize the article: tokens
tokens = word_tokenize(article)
tokens

["''",
 "'Debugging",
 "''",
 "'",
 'is',
 'the',
 'process',
 'of',
 'finding',
 'and',
 'resolving',
 'of',
 'defects',
 'that',
 'prevent',
 'correct',
 'operation',
 'of',
 'computer',
 'software',
 'or',
 'a',
 'system',
 '.',
 'Numerous',
 'books',
 'have',
 'been',
 'written',
 'about',
 'debugging',
 '(',
 'see',
 'below',
 ':',
 '#',
 'Further',
 'reading|Further',
 'reading',
 ')',
 ',',
 'as',
 'it',
 'involves',
 'numerous',
 'aspects',
 ',',
 'including',
 'interactive',
 'debugging',
 ',',
 'control',
 'flow',
 ',',
 'integration',
 'testing',
 ',',
 'Logfile|log',
 'files',
 ',',
 'monitoring',
 '(',
 'Application',
 'monitoring|application',
 ',',
 'System',
 'Monitoring|system',
 ')',
 ',',
 'memory',
 'dumps',
 ',',
 'Profiling',
 '(',
 'computer',
 'programming',
 ')',
 '|profiling',
 ',',
 'Statistical',
 'Process',
 'Control',
 ',',
 'and',
 'special',
 'design',
 'tactics',
 'to',
 'improve',
 'detection',
 'while',
 'simplifying',
 'changes',
 '.',
 'Origin',
 'A

In [110]:
# Convert the tokens into lowercase: lower_tokens
lower_tokens = [t.lower() for t in tokens]
lower_tokens

["''",
 "'debugging",
 "''",
 "'",
 'is',
 'the',
 'process',
 'of',
 'finding',
 'and',
 'resolving',
 'of',
 'defects',
 'that',
 'prevent',
 'correct',
 'operation',
 'of',
 'computer',
 'software',
 'or',
 'a',
 'system',
 '.',
 'numerous',
 'books',
 'have',
 'been',
 'written',
 'about',
 'debugging',
 '(',
 'see',
 'below',
 ':',
 '#',
 'further',
 'reading|further',
 'reading',
 ')',
 ',',
 'as',
 'it',
 'involves',
 'numerous',
 'aspects',
 ',',
 'including',
 'interactive',
 'debugging',
 ',',
 'control',
 'flow',
 ',',
 'integration',
 'testing',
 ',',
 'logfile|log',
 'files',
 ',',
 'monitoring',
 '(',
 'application',
 'monitoring|application',
 ',',
 'system',
 'monitoring|system',
 ')',
 ',',
 'memory',
 'dumps',
 ',',
 'profiling',
 '(',
 'computer',
 'programming',
 ')',
 '|profiling',
 ',',
 'statistical',
 'process',
 'control',
 ',',
 'and',
 'special',
 'design',
 'tactics',
 'to',
 'improve',
 'detection',
 'while',
 'simplifying',
 'changes',
 '.',
 'origin',
 'a

In [111]:
# Create a Counter with the lowercase tokens: bow_simple
bow_simple = Counter(lower_tokens)
bow_simple

Counter({"''": 69,
         "'debugging": 1,
         "'": 2,
         'is': 25,
         'the': 150,
         'process': 12,
         'of': 81,
         'finding': 1,
         'and': 41,
         'resolving': 1,
         'defects': 3,
         'that': 14,
         'prevent': 1,
         'correct': 1,
         'operation': 2,
         'computer': 12,
         'software': 16,
         'or': 25,
         'a': 60,
         'system': 19,
         '.': 89,
         'numerous': 2,
         'books': 1,
         'have': 7,
         'been': 3,
         'written': 1,
         'about': 3,
         'debugging': 39,
         '(': 40,
         'see': 5,
         'below': 1,
         ':': 31,
         '#': 1,
         'further': 1,
         'reading|further': 1,
         'reading': 1,
         ')': 40,
         ',': 151,
         'as': 21,
         'it': 18,
         'involves': 1,
         'aspects': 1,
         'including': 1,
         'interactive': 3,
         'control': 4,
         'flow': 2,
  

In [112]:
# Print the 10 most common tokens
print(bow_simple.most_common(10))

[(',', 151), ('the', 150), ('.', 89), ('of', 81), ("''", 69), ('to', 63), ('a', 60), ('``', 47), ('in', 44), ('and', 41)]


## 0.2.4 Text preprocessing steps

Which of the following are useful text preprocessing steps?

R:/ Lemmatization, lowercasing, removing unwanted tokens.

## 0.2.5 Text preprocessing practice

In [113]:
# Retain alphabetic words: alpha_only
alpha_only = [t for t in lower_tokens if t.isalpha()]
alpha_only

['is',
 'the',
 'process',
 'of',
 'finding',
 'and',
 'resolving',
 'of',
 'defects',
 'that',
 'prevent',
 'correct',
 'operation',
 'of',
 'computer',
 'software',
 'or',
 'a',
 'system',
 'numerous',
 'books',
 'have',
 'been',
 'written',
 'about',
 'debugging',
 'see',
 'below',
 'further',
 'reading',
 'as',
 'it',
 'involves',
 'numerous',
 'aspects',
 'including',
 'interactive',
 'debugging',
 'control',
 'flow',
 'integration',
 'testing',
 'files',
 'monitoring',
 'application',
 'system',
 'memory',
 'dumps',
 'profiling',
 'computer',
 'programming',
 'statistical',
 'process',
 'control',
 'and',
 'special',
 'design',
 'tactics',
 'to',
 'improve',
 'detection',
 'while',
 'simplifying',
 'changes',
 'origin',
 'a',
 'computer',
 'log',
 'entry',
 'from',
 'the',
 'mark',
 'nbsp',
 'ii',
 'with',
 'a',
 'moth',
 'taped',
 'to',
 'the',
 'page',
 'the',
 'terms',
 'bug',
 'and',
 'debugging',
 'are',
 'popularly',
 'attributed',
 'to',
 'admiral',
 'grace',
 'hopper',
 '

In [114]:
# Remove all stop words: no_stops
no_stops = [t for t in alpha_only if t not in stopwords.words('english')]
no_stops

['process',
 'finding',
 'resolving',
 'defects',
 'prevent',
 'correct',
 'operation',
 'computer',
 'software',
 'system',
 'numerous',
 'books',
 'written',
 'debugging',
 'see',
 'reading',
 'involves',
 'numerous',
 'aspects',
 'including',
 'interactive',
 'debugging',
 'control',
 'flow',
 'integration',
 'testing',
 'files',
 'monitoring',
 'application',
 'system',
 'memory',
 'dumps',
 'profiling',
 'computer',
 'programming',
 'statistical',
 'process',
 'control',
 'special',
 'design',
 'tactics',
 'improve',
 'detection',
 'simplifying',
 'changes',
 'origin',
 'computer',
 'log',
 'entry',
 'mark',
 'nbsp',
 'ii',
 'moth',
 'taped',
 'page',
 'terms',
 'bug',
 'debugging',
 'popularly',
 'attributed',
 'admiral',
 'grace',
 'hopper',
 'http',
 'grace',
 'hopper',
 'foldoc',
 'working',
 'harvard',
 'mark',
 'ii',
 'computer',
 'harvard',
 'university',
 'associates',
 'discovered',
 'moth',
 'stuck',
 'relay',
 'thereby',
 'impeding',
 'operation',
 'whereupon',
 'remark

In [115]:
# Instantiate the WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Lemmatize all tokens into a new list: lemmatized
lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in no_stops]
lemmatized

['process',
 'finding',
 'resolving',
 'defect',
 'prevent',
 'correct',
 'operation',
 'computer',
 'software',
 'system',
 'numerous',
 'book',
 'written',
 'debugging',
 'see',
 'reading',
 'involves',
 'numerous',
 'aspect',
 'including',
 'interactive',
 'debugging',
 'control',
 'flow',
 'integration',
 'testing',
 'file',
 'monitoring',
 'application',
 'system',
 'memory',
 'dump',
 'profiling',
 'computer',
 'programming',
 'statistical',
 'process',
 'control',
 'special',
 'design',
 'tactic',
 'improve',
 'detection',
 'simplifying',
 'change',
 'origin',
 'computer',
 'log',
 'entry',
 'mark',
 'nbsp',
 'ii',
 'moth',
 'taped',
 'page',
 'term',
 'bug',
 'debugging',
 'popularly',
 'attributed',
 'admiral',
 'grace',
 'hopper',
 'http',
 'grace',
 'hopper',
 'foldoc',
 'working',
 'harvard',
 'mark',
 'ii',
 'computer',
 'harvard',
 'university',
 'associate',
 'discovered',
 'moth',
 'stuck',
 'relay',
 'thereby',
 'impeding',
 'operation',
 'whereupon',
 'remarked',
 'de

In [116]:
# Create the bag-of-words: bow
bow = Counter(lemmatized)
bow

Counter({'process': 13,
         'finding': 1,
         'resolving': 1,
         'defect': 4,
         'prevent': 1,
         'correct': 1,
         'operation': 2,
         'computer': 14,
         'software': 16,
         'system': 25,
         'numerous': 2,
         'book': 2,
         'written': 1,
         'debugging': 39,
         'see': 5,
         'reading': 1,
         'involves': 1,
         'aspect': 1,
         'including': 1,
         'interactive': 3,
         'control': 4,
         'flow': 2,
         'integration': 1,
         'testing': 4,
         'file': 3,
         'monitoring': 1,
         'application': 2,
         'memory': 5,
         'dump': 4,
         'profiling': 1,
         'programming': 9,
         'statistical': 1,
         'special': 2,
         'design': 4,
         'tactic': 1,
         'improve': 1,
         'detection': 2,
         'simplifying': 1,
         'change': 5,
         'origin': 2,
         'log': 1,
         'entry': 2,
         'mark':

In [117]:
# Print the 10 most common tokens
print(bow.most_common(10))

[('debugging', 39), ('system', 25), ('bug', 17), ('software', 16), ('problem', 15), ('tool', 15), ('computer', 14), ('process', 13), ('term', 13), ('debugger', 13)]


## 0.2.8 What are word vectors?

What are word vectors and how do they help with NLP?

R:/ Word vectors are multi-dimensional mathematical representations of words created using deep learning methods. They give us insight into relationships between words in a corpus.

## 0.2.9 Creating and querying a corpus with gensim

In [118]:
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(articles)
dictionary

In [119]:
# Select the id for "computer": computer_id
computer_id = dictionary.token2id.get("computer")
computer_id

99

In [120]:
# Use computer_id with the dictionary to print the word
print(dictionary.get(computer_id))

computer


In [121]:
# Create a MmCorpus: corpus
corpus = [dictionary.doc2bow(article) for article in articles]
corpus

[[(0, 20),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 12),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 2),
  (35, 1),
  (36, 5),
  (37, 1),
  (38, 1),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 2),
  (43, 1),
  (44, 1),
  (45, 15),
  (46, 6),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 2),
  (55, 1),
  (56, 2),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 6),
  (66, 2),
  (67, 1),
  (68, 2),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 2),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 4),
  (79, 1),
  (80, 1),
  (81, 3),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 3),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91,

In [122]:
# Print the first 10 word ids with their frequency counts from the fifth document
print(corpus[4][:10])

[(0, 51), (5, 3), (23, 7), (28, 1), (30, 1), (34, 2), (36, 7), (43, 4), (58, 1), (67, 1)]


## 0.2.10 Gensim bag-of-words

In [123]:
# Save the fifth document: doc
doc = corpus[4]
doc

[(0, 51),
 (5, 3),
 (23, 7),
 (28, 1),
 (30, 1),
 (34, 2),
 (36, 7),
 (43, 4),
 (58, 1),
 (67, 1),
 (69, 2),
 (81, 9),
 (85, 1),
 (89, 1),
 (90, 1),
 (93, 1),
 (94, 1),
 (99, 2),
 (102, 1),
 (107, 3),
 (109, 1),
 (112, 1),
 (118, 5),
 (120, 1),
 (122, 3),
 (129, 1),
 (134, 3),
 (135, 1),
 (136, 2),
 (138, 2),
 (140, 2),
 (146, 3),
 (148, 1),
 (153, 1),
 (159, 1),
 (162, 35),
 (166, 1),
 (170, 2),
 (173, 1),
 (178, 1),
 (186, 1),
 (194, 1),
 (196, 1),
 (199, 1),
 (201, 3),
 (206, 1),
 (212, 1),
 (216, 1),
 (218, 3),
 (220, 1),
 (226, 3),
 (235, 1),
 (238, 1),
 (261, 1),
 (273, 1),
 (275, 2),
 (278, 2),
 (280, 1),
 (282, 1),
 (284, 6),
 (286, 1),
 (299, 1),
 (302, 1),
 (303, 1),
 (304, 1),
 (305, 1),
 (307, 1),
 (317, 5),
 (318, 2),
 (322, 2),
 (325, 3),
 (327, 1),
 (329, 1),
 (331, 1),
 (332, 1),
 (335, 2),
 (337, 1),
 (338, 1),
 (344, 1),
 (354, 1),
 (357, 2),
 (358, 1),
 (369, 1),
 (370, 2),
 (373, 1),
 (378, 1),
 (392, 1),
 (399, 1),
 (404, 1),
 (408, 15),
 (412, 3),
 (413, 2),
 (415

In [124]:
# Sort the doc for frequency: bow_doc
bow_doc = sorted(doc, key=lambda w: w[1], reverse=True)
bow_doc

[(0, 51),
 (162, 35),
 (1536, 28),
 (408, 15),
 (1114, 11),
 (1465, 10),
 (81, 9),
 (429, 9),
 (651, 9),
 (477, 8),
 (1694, 8),
 (23, 7),
 (36, 7),
 (851, 7),
 (1928, 7),
 (284, 6),
 (481, 6),
 (1885, 6),
 (118, 5),
 (317, 5),
 (430, 5),
 (694, 5),
 (1735, 5),
 (1931, 5),
 (43, 4),
 (416, 4),
 (469, 4),
 (567, 4),
 (1435, 4),
 (1742, 4),
 (1757, 4),
 (1861, 4),
 (1868, 4),
 (5, 3),
 (107, 3),
 (122, 3),
 (134, 3),
 (146, 3),
 (201, 3),
 (218, 3),
 (226, 3),
 (325, 3),
 (412, 3),
 (504, 3),
 (591, 3),
 (750, 3),
 (757, 3),
 (823, 3),
 (838, 3),
 (854, 3),
 (873, 3),
 (931, 3),
 (1140, 3),
 (1748, 3),
 (1752, 3),
 (1790, 3),
 (1804, 3),
 (1818, 3),
 (1821, 3),
 (1884, 3),
 (1888, 3),
 (1905, 3),
 (1993, 3),
 (1998, 3),
 (34, 2),
 (69, 2),
 (99, 2),
 (136, 2),
 (138, 2),
 (140, 2),
 (170, 2),
 (275, 2),
 (278, 2),
 (318, 2),
 (322, 2),
 (335, 2),
 (357, 2),
 (370, 2),
 (413, 2),
 (426, 2),
 (448, 2),
 (482, 2),
 (492, 2),
 (503, 2),
 (552, 2),
 (618, 2),
 (632, 2),
 (668, 2),
 (702, 2),
 

In [125]:
# Print the top 5 words of the document alongside the count
for word_id, word_count in bow_doc[:5]:
    print(dictionary.get(word_id), word_count)

'' 51
engineering 35
reverse 28
software 15
ref 11


In [128]:
# Create the defaultdict: total_word_count
total_word_count = defaultdict(int)
for word_id, word_count in itertools.chain.from_iterable(corpus):
    total_word_count[word_id] += word_count
total_word_count

defaultdict(int,
            {0: 322,
             1: 1,
             2: 1,
             3: 1,
             4: 1,
             5: 47,
             6: 2,
             7: 1,
             8: 1,
             9: 1,
             10: 1,
             11: 1,
             12: 1,
             13: 1,
             14: 3,
             15: 2,
             16: 1,
             17: 13,
             18: 3,
             19: 1,
             20: 1,
             21: 1,
             22: 1,
             23: 186,
             24: 5,
             25: 2,
             26: 1,
             27: 1,
             28: 3,
             29: 7,
             30: 4,
             31: 1,
             32: 2,
             33: 4,
             34: 8,
             35: 1,
             36: 42,
             37: 1,
             38: 1,
             39: 4,
             40: 2,
             41: 5,
             42: 10,
             43: 6,
             44: 6,
             45: 27,
             46: 13,
             47: 2,
             48: 1,
   

In [129]:
# Create a sorted list from the defaultdict: sorted_word_count
sorted_word_count = sorted(total_word_count.items(), key=lambda w: w[1], reverse=True) 
sorted_word_count

[(0, 322),
 (23, 186),
 (408, 153),
 (99, 151),
 (81, 97),
 (1114, 87),
 (364, 70),
 (250, 68),
 (854, 64),
 (429, 63),
 (477, 60),
 (107, 52),
 (188, 49),
 (85, 48),
 (1465, 48),
 (5, 47),
 (641, 47),
 (284, 46),
 (692, 44),
 (162, 43),
 (36, 42),
 (280, 39),
 (251, 38),
 (1536, 37),
 (272, 35),
 (723, 33),
 (102, 31),
 (600, 31),
 (724, 31),
 (1286, 31),
 (206, 30),
 (430, 30),
 (173, 29),
 (185, 29),
 (475, 29),
 (640, 29),
 (318, 28),
 (45, 27),
 (601, 27),
 (931, 27),
 (210, 26),
 (1187, 26),
 (124, 25),
 (317, 25),
 (450, 25),
 (122, 24),
 (136, 24),
 (287, 24),
 (481, 24),
 (493, 24),
 (687, 23),
 (719, 23),
 (365, 22),
 (1592, 22),
 (321, 21),
 (504, 21),
 (467, 20),
 (478, 19),
 (1164, 19),
 (633, 18),
 (651, 18),
 (715, 18),
 (851, 18),
 (211, 17),
 (283, 17),
 (307, 17),
 (357, 17),
 (968, 17),
 (4017, 17),
 (135, 16),
 (226, 16),
 (392, 16),
 (415, 16),
 (505, 16),
 (330, 15),
 (2312, 15),
 (160, 14),
 (325, 14),
 (639, 14),
 (688, 14),
 (693, 14),
 (721, 14),
 (730, 14),
 

In [130]:
# Print the top 5 words across all documents alongside the count
for word_id, word_count in sorted_word_count[:5]:
    print(dictionary.get(word_id), word_count)

'' 322
`` 186
software 153
computer 151
cite 97


## 0.2.12 What is tf-idf?

You want to calculate the tf-idf weight for the word "computer", which appears five times in a document containing 100 words. Given a corpus containing 200 documents, with 20 documents mentioning the word "computer", tf-idf can be calculated by multiplying term frequency with inverse document frequency.

Term frequency = percentage share of the word compared to all tokens in the document Inverse document frequency = logarithm of the total number of documents in a corpora divided by the number of documents containing the term

Which of the below options is correct?

R:/ (5 / 100) * log(200 / 20)

## 0.2.13 Tf-idf with Wikipedia

In [127]:
print('ok_')

ok_
